In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
71,"Hugh (Ed Harris) is a hotshot, bachelor senato...",positive
797,There is nothing unique in either the TV Serie...,negative
590,"Twenty five years ago, I showed this film in s...",positive
369,Guys and Dolls is a unique play based on the c...,positive
711,"Even if it were remotely funny, this mouldy wa...",negative


In [3]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Satvikk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Satvikk\AppData\Local\Temp\ipykernel_1964\3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [5]:
df = normalize_text(df)
df.head()

,review,sentiment
71,hugh ed harris hotshot bachelor senator determ...,positive
797,nothing unique either tv series movie prequel ...,negative
590,twenty five year ago showed film child s class...,positive
369,guy doll unique play based character sky maste...,positive
711,even remotely funny mouldy waxwork film would ...,negative


In [6]:
df['sentiment'].value_counts()

sentiment
negative    263
positive    237
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
71,hugh ed harris hotshot bachelor senator determ...,1
797,nothing unique either tv series movie prequel ...,0
590,twenty five year ago showed film child s class...,1
369,guy doll unique play based character sky maste...,1
711,even remotely funny mouldy waxwork film would ...,0


In [10]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [13]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/satvikk/Capstone.mlflow')
dagshub.init(repo_owner='satvikk', repo_name='Capstone', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

d:\Code\NewProj\venv\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=fb1248e8-3539-4383-a1d6-8e2bcde1626d&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=25fedd2ddd5e9e28243292abde589bfad92ec4acc2beea44e3c7f462e0c95cff




Accessing as satvikk

Initialized MLflow to track repo "satvikk/Capstone"

Repository satvikk/Capstone initialized!

2025/03/25 10:49:05 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/d7a455d7a4f34cf38c6339e387acbc86', creation_time=1742879945340, experiment_id='0', last_update_time=1742879945340, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [14]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-03-25 10:51:25,170 - INFO - Starting MLflow run...
2025-03-25 10:51:26,755 - INFO - Logging preprocessing parameters...
2025-03-25 10:51:28,135 - INFO - Initializing Logistic Regression model...
2025-03-25 10:51:28,135 - INFO - Fitting the model...
2025-03-25 10:51:28,166 - INFO - Model training complete.
2025-03-25 10:51:28,167 - INFO - Logging model parameters...
2025-03-25 10:51:28,577 - INFO - Making predictions...
2025-03-25 10:51:28,577 - INFO - Calculating evaluation metrics...
2025-03-25 10:51:28,589 - INFO - Logging evaluation metrics...
2025-03-25 10:51:30,620 - INFO - Saving and logging the model...
2025/03/25 10:51:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-03-25 10:51:43,325 - INFO - Model training and logging completed in 16.57 seconds.
2025-03-25 10:51:43,328 - INFO - Accuracy: 0.6
2025-03-25 10:51:43,328 - INFO - Precision: 0.

🏃 View run brawny-swan-900 at: https://dagshub.com/satvikk/Capstone.mlflow/#/experiments/0/runs/5d62f7dac9cd4a30ab1999680d4fd229
🧪 View experiment at: https://dagshub.com/satvikk/Capstone.mlflow/#/experiments/0
